In [31]:
import numpy as np
import cv2
import os
import glob
import json
import matplotlib.pyplot as plt

## Read list of images and mask files

In [24]:
imglist = glob.glob("rs19_val/jpgs/rs19_val/*.jpg")
jsonlist = []
for i in imglist:
    jsonlist.append(i[0:9] + 'jsons' + i[-21:-3] + 'json')

## Mark tracks

Selection:

- all tracks
- ego track at value 255, other tracks at differen level (NonEgo)
- show plots
- save overlay images

In [167]:
ego = True # Mark only ego tracks
NonEgo = 0
plot = False # Show plots for verification
overlay = False
for j in jsonlist:
    # for all images
    with open(j) as json_file:
        # read json
        data = json.load(json_file)
        # open image
        im = cv2.imread('rs19_val/jpgs/rs19_val/' + data['frame'] + '.jpg')
        # read image dimensions
        w = data['imgWidth']
        h = data['imgHeight']
        # create mask image
        mask = np.zeros((h, w))
        for o in data['objects']:
            if o['label'] == 'rail':
                if 'polyline-pair' in o:
                    if ego: # Label only ego track
                        centerw = 0.5*(np.array(o['polyline-pair'][0])[0,0] + np.array(o['polyline-pair'][1])[0,0])
                        centerh = np.array(o['polyline-pair'][0])[0,1]
                        if (centerw > 0.4*w) & (centerw < 0.6*w) & (centerh > 0.8*h):
                            # Ego track found, label only this
                            arr = np.concatenate((np.array(o['polyline-pair'][0]), 
                                         np.flip(np.array(o['polyline-pair'][1]), axis = 0)))
                            arr = np.append(arr, np.array(o['polyline-pair'][0])[0:1,:], axis = 0)
                    #print(o['polyline-pair'][1])
                            cv2.fillPoly(mask, [arr], color=(255))
                        # Label all rails
                        else:
                            arr = np.concatenate((np.array(o['polyline-pair'][0]), 
                                         np.flip(np.array(o['polyline-pair'][1]), axis = 0)))
                            arr = np.append(arr, np.array(o['polyline-pair'][0])[0:1,:], axis = 0)
                    #print(o['polyline-pair'][1])
                            cv2.fillPoly(mask, [arr], color=(NonEgo))
                    else:
                        arr = np.concatenate((np.array(o['polyline-pair'][0]), 
                                         np.flip(np.array(o['polyline-pair'][1]), axis = 0)))
                        arr = np.append(arr, np.array(o['polyline-pair'][0])[0:1,:], axis = 0)
                    #print(o['polyline-pair'][1])
                        cv2.fillPoly(mask, [arr], color=(255))
        if plot:
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 3, 1)
            plt.imshow(mask)
            plt.subplot(1, 3, 2)
            plt.imshow(im)
            plt.subplot(1, 3, 3)
            plt.imshow(np.add(mask,[0.33*im[:,:,0]+ 0.33*im[:,:,1] + 0.33*im[:,:,2]])[0,:,:])
        if (mask > 0).any():
            # Save mask only if there is at least on track
            cv2.imwrite('rs19_val/masks/'+ data['frame'] + '_Annotation.png', mask)
            if overlay:
                cv2.imwrite('rs19_val/overlays/'+ data['frame'] + '_Overlay.png', 
                            np.add(mask,[0.33*im[:,:,0]+ 0.33*im[:,:,1] + 0.33*im[:,:,2]])[0,:,:])
        